In [1]:
from tinc import *

In [2]:
%pylab inline

Populating the interactive namespace from numpy and matplotlib


C:\Users\Andres\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.9_qbz5n2kfra8p0\LocalCache\local-packages\Python39\site-packages\IPython\core\magics\pylab.py:159: UserWarning: pylab import has clobbered these variables: ['unique', 'interactive']
`%matplotlib` prevents importing * from pylab and numpy
  warn("pylab import has clobbered these variables: %s"  % clobbered +


This notebook relies on the tinc-python unit test data folders. Clone the tinc-python repo from https://github.com/AlloSphere-Research-Group/tinc-python

In [3]:
# Adjust this path to point to the location of tinc-python in your system
tinc_path = "../"

To create a data pool, the first thing needed is a parameter space that defines the filesystem configuration of the data.

In [4]:
import os
os.listdir(tinc_path + "tinc/tests/data")

['folder1', 'folder2', 'folder3']

All of these folders contain a file with the same name:

In [5]:
import os
os.listdir(tinc_path + "tinc/tests/data/folder1")

['results.json']

All these json files contain 3 fields with 8 elements each:

In [6]:
with open(tinc_path + "tinc/tests/data/folder1/results.json") as f:
    print(json.load(f))

{'field1': [0, 1, 2, 3, 4, 5, 6, 7], 'field2': [1, 2, 3, 4, 5, 6, 7, 8], 'field3': [4, 5, 6, 7, 8, 9, 0, 1]}


In [7]:
param1 = Parameter("param1")
param1.values = [0.1, 0.2, 0.3] # Arbitrarily defined here, but in a real dataset would represent the parameter values that map to folders
param1.ids = ["folder1", "folder2", "folder3"]

A second parameter can map to the internal "columns" in the fields within the results.json file:

In [8]:
inner_param = Parameter("inner_param")
inner_param.values = linspace(0.1, 0.8, 8)


In [9]:
ps = ParameterSpace()
ps.register_parameters([param1, inner_param])
ps.set_root_path(tinc_path)
ps.set_current_path_template("tinc/tests/data/%%param1:ID%%")

To test things are working:

In [10]:
def print_current_path(discard_value = 0):
    print(ps.get_current_relative_path())
param1.register_callback(print_current_path)
param1.interactive_widget()

interactive(children=(FloatSlider(value=0.1, description='param1', max=0.3, min=0.1, readout_format='.7f', ste…

In [11]:
ps.get_root_path()

'../'

In [12]:
dp = DataPoolJson("data", ps)
dp.register_data_file("results.json", "inner_param")
dp.get_current_files()

['c:\\Users\\Andres\\source\\repos\\casm_viewer\\external\\tinc\\tinc-python\\tinc\\tests\\data\\folder1\\results.json']

In [13]:
dp.list_fields()

['field1', 'field2', 'field3']

In [17]:
dp.get_slice("field1","inner_param")

masked_array(data=[0., 1., 2., 3., 4., 5., 6., 7.],
             mask=False,
       fill_value=1e+20,
            dtype=float32)

In [18]:
param1.value = 0.2
dp.get_slice("field1","inner_param")

tinc/tests/data/folder2


masked_array(data=[1., 2., 3., 4., 5., 6., 7., 8.],
             mask=False,
       fill_value=1e+20,
            dtype=float32)

Because param1 is now 0.2, the folder is "folder2":

In [19]:
with open(tinc_path + "tinc/tests/data/folder2/results.json") as f:
    print(json.load(f))

{'field1': [1, 2, 3, 4, 5, 6, 7, 8], 'field2': [0, 1, 2, 3, 4, 5, 6, 7], 'field3': [5, 7, 8, 9, 0, 1, 6, 4]}


We can also slice across directories by slicing across the "param1" dimension:

In [20]:
dp.get_slice("field1","param1")

masked_array(data=[2., 3., 8.],
             mask=False,
       fill_value=1e+20,
            dtype=float32)

In [21]:
inner_param.value = 0.3
dp.get_slice("field1","param1")

masked_array(data=[2., 3., 8.],
             mask=False,
       fill_value=1e+20,
            dtype=float32)

In [22]:
inner_param.value = 0.1
dp.get_slice("field1","param1")

masked_array(data=[0., 1., 5.],
             mask=False,
       fill_value=1e+20,
            dtype=float32)

## Data pools with multiple file system parameters

In [23]:
os.listdir(tinc_path + "tinc/tests/data2")

['folderA_1', 'folderA_2', 'folderB_1', 'folderB_2']

In this dataset, two parameters determine the directory. One parameter determines the prefix (folderA/folderB) and another parameter the suffix (_1, _2). Notice that the number of directories must be the product of the two parameters' sizes.

In [24]:
with open(tinc_path + "tinc/tests/data2/folderA_1/results.json") as f:
    print(json.load(f))

{'field1': [0, 1, 2, 3, 4, 5, 6, 7], 'field2': [1, 2, 3, 4, 5, 6, 7, 8], 'field3': [4, 5, 6, 7, 8, 9, 0, 1]}


In [30]:
param1 = Parameter("param1")
param1.values = [0.1, 0.1, 0.2, 0.2]
param1.ids = ["folderA_1", "folderA_2", "folderB_1", "folderB_2"]

param2 = Parameter("param2")
param2.values = [1,1,2,2]
param2.ids = ["folderA_1", "folderB_1", "folderA_2", "folderB_2"]

inner_param = Parameter("inner_param")
inner_param.values = linspace(0.1, 0.8, 8)

In [31]:
ps = ParameterSpace()
ps.register_parameters([param1, param2, inner_param])
ps.set_root_path(tinc_path)
ps.set_current_path_template("tinc/tests/data/%%param1,param2%%")

In [32]:
ps.get_current_relative_path()

'tinc/tests/data/folderA_1'

In [35]:
param1.value = 0.2
param2.value = 1
ps.get_current_relative_path()

'tinc/tests/data/folderB_1'

In [34]:
ps.get_common_id([param1, param2])

'folderB_1'